In [1]:
import os
import pyvista as pv
from math import dist
import numpy as np

In [2]:
def outRayFinder(pts):
        if len(pts) == 0:
                return None, None
            # Find the first element in pts that is differnt from pts[0] and return that point onelin
        if len(pts) > 2:
            points = pts
            # Create a unique list of 3d points 
            points = np.unique(pts, axis=0)
            # Bug here with the idx bit
            check = np.where(np.isclose(points, points[0])  != [True, True, True])[0]
            leng = len(check)
            if leng < 1:
                return None, None
            if leng == 1:
                idx = check[0]
            elif leng == 2:
                idx = check[1]
            elif leng >= 3:
                idx = check[round(leng/2)]
            return pts[0], pts[idx-1] # This does have some slight issues given a parallel intersection and no easy way of verifying if its an out ray or not so we naively return the last point

        elif len(pts) == 2:
            return pts[0], pts[1]
        else:
            return None,None

In [3]:
DATA_DIR = "data/cow_mesh"
rotation = 75
end = (15, 1.4, -0.2)
end2 = (3, -.6, 0.4)

camera = pv.Camera()
# Camera Pos X(-5.0 to 5.0), Y(0.0 to 2.0), Z(0.0 to 5.0)
camera.position =(-10, 0, 0)

near_range = 0.3
far_range = 10
camera.clipping_range = (near_range, far_range)
c_pos = np.array(camera.position)

frustum = camera.view_frustum().copy()

obj_texture = pv.read_texture(os.path.join(DATA_DIR, "cow_texture.png"))
obj_mesh = pv.read(os.path.join(DATA_DIR, "cow.obj"),)
obj_mesh.rotate_vector(vector=(0, 1, 0), angle=rotation, inplace=True)

line = pv.Line(pointa=camera.position, pointb=end, resolution= 1)
line2 = pv.Line(pointa=camera.position, pointb=end2, resolution= 1)

points1, ind1 = obj_mesh.ray_trace(camera.position, end)
points2, ind1 = obj_mesh.ray_trace(camera.position, end2)

in1, out1, in2, out2 = None, None, None, None

# in1, out1 = outRayFinder(points1)
in2, out2 = outRayFinder(points2)

pl = pv.Plotter()

_ = pl.add_mesh(obj_mesh, texture=obj_texture,opacity=0.55)
# _ = pl.add_mesh(line, color='red')
_ = pl.add_mesh(line2, color='blue')
# _ = pl.add_mesh(frustum, style="wireframe",color="orange")
_ = pl.add_points(np.array(camera.position), color='white', point_size=15)
if in1 is not None:
    _ = pl.add_points(in1, color='red', point_size=25)
    _ = pl.add_points(out1, color='red', point_size=25)
    print("Red Thickness:", dist(in1, out1))
if in2 is not None:
    _ = pl.add_points(in2, color='blue', point_size=25)
    _ = pl.add_points( out2, color='blue', point_size=25)
    print("Blue Thickness:", dist(in2, out2))
pl.camera = camera
pl.show(jupyter_backend='pythreejs')

Blue Thickness: 0.31062223212652323


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [4]:
pl.show_bounds(grid='back', location='outer', all_edges=True)

(vtkmodules.vtkRenderingAnnotation.vtkCubeAxesActor)000001C719A4CCA0